# Introduction 

the goal of this notebook is to train a custom object detection model for cars, overall the following steps are implemented:
* First the desired object categories are detected
* second whether the objects are colored red or not is estimated 

the inference result is then saved as a CSV file.

#Guide on how to use the notebook

using the notebook is very straight forward and the most important thing is to mount your google drive and **create the proper directory according** to the following:

after mounting your google drive please creat the following directory structure under '/content/drive/MyDrive' :

        objectDetection/
               data/
                   output/
                   rawDataset/
                   test/
                   train/
                   valid/
                   customTestDataset/
                    
               trainedModel/
                   coco_eval
                   eval/
                   savedModles/
                   

This will be the directory where we are going to have our raw data as well as training validation and test set.

Following is the explanation for each directory folder:
explaining each directory:

* 1-"output" is the directory where the final CSV is stored
* 2- "rawDataset" is where we put the provided dataset (images+json file)
* 3- "test" is where the test data set is created after the function prepare_data_sets() is executed 
* 4- "train" is where the training data set is created after the function prepare_data_sets() is executed 
* 5- "valid" is where the validation data set is created after the function prepare_data_sets() is executed 
* 6- traineModel is where the final model is stored after completing the training job
it's important to note that if you already have a trained model and don't want to run the training job again
copy your model here and comment out the training and evaluation lines in the main
* 7- saved models is where you can save your trained models
* 8- eval is where evaluation data is saved after running the evaluation job
* 9- customTestDataset contains a dataset for further testing that i generated using roboflow

Therefore do the following: 
* upload the dataset into the "rawDataset" folder
* if you don't want to train the model (it takes a lot of time) copy the model file from the "savedModles" folder

**note** that the zip file already has all the files outputs and models (because i ran the entire pipeline first then saved the zipfile)

that is it. now you can run the code cells in the notebook

this is a link to all the project files with a similar directory structure:

https://drive.google.com/file/d/1T6ACsLNld-NXYaiL8181qJydvYlu6SPi/view?usp=sharing

**note** if you dont want to train the model (already have the trained model) dont execute the "start training" , "tensorboard" and "start evaluating the model trained" code cells

##Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Detectron2 Dependencies

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

## Importing the dependancies

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
 
# Setup detectron2 logger
import detectron2
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
 
# import some common libraries
import json
import shutil
import os
import pandas as pd
import numpy as np
import cv2
import random
import csv
import pathlib
import glob
import random
from google.colab.patches import cv2_imshow

# Data preprocessing and preparation

this block will take the images and the associated Json file (in coco format) and creats three sets of data. a set for training a set for validation and a set for testing.

**Note** that the datasets will be stored in the test, train, and valid folders 

In [ ]:
cfg = get_cfg()
 
mainPath =  '/content/drive/MyDrive/objectDetection/'
filedir = os.path.join(mainPath,'data','rawDataSet','_annotations.coco.json' );
directory = os.path.join(mainPath,'data','rawDataSet')
 
imList=[];

In [ ]:
   
file = open(filedir)
 
#annotation data that is provided 
annData = json.load(file);
                  
annListFlt= annData["annotations"]      
annDataflt = {"annotations": annListFlt}

##### dividing to validation test and train 
 
length = len(annListFlt)
 
# determining the traing validation and test percentages
pt=0.7
pv=0.2
 
tre = int(length*pt); #end of train
vs = tre; #start of vallidation
ve = vs+ int(length*pv);# end of validation
ts= ve; #start of test
te = length; #end of test 
 
imageTrain=[]
imageValid=[]
imageTest=[]
 
trainSet=annData["images"][0:tre];
validSet=annData["images"][vs:ve];
testSet=annData["images"][ts:te];

trainSetann=annData["annotations"][0:tre];
validSetann=annData["annotations"][vs:ve];
testSetann=annData["annotations"][ts:te];
 
###copying test files to test folder
path=[];
for diction in testSet:
    path.append(os.path.join(directory,str(diction["file_name"])))
for f in path:
    shutil.copy(f, os.path.join(mainPath,'data','test'))
 
################copying the files for training
trainPath=[];
for diction in trainSet:
    trainPath.append(os.path.join(directory,str(diction["file_name"])))
for f in trainPath:
    shutil.copy(f, os.path.join(mainPath,'data','train'))
 
###############copying the files for validation
validPath=[];
for diction in validSet:
    validPath.append(os.path.join(directory,str(diction["file_name"])))
for f in validPath:
    shutil.copy(f, os.path.join(mainPath,'data','valid')) 
 
#bulding the json files for each set 
CocoFileTrain = {"categories": annData["categories"] , "images":trainSet ,"annotations": trainSetann };
CocoFileValid = {"categories": annData["categories"] , "images":validSet ,"annotations": validSetann };
CocoFileTest = {"categories": annData["categories"] , "images":testSet ,"annotations": testSetann };
 
 
#########saving the json files
with open(os.path.join(mainPath,'data','train','CocoAnnTrain.json'), 'w') as fp1:
    json.dump(CocoFileTrain, fp1)
with open(os.path.join(mainPath,'data','valid','CocoAnnValid.json'), 'w') as fp2:
    json.dump(CocoFileValid, fp2)
with open(os.path.join(mainPath,'data','test','CocoAnnTest.json'), 'w') as fp3:
    json.dump(CocoFileTest, fp3)

#Registering the datasets

here we will register the generated data sets in the previous step

In [ ]:
register_coco_instances("my_dataset_train", {}, os.path.join(mainPath,'data','train','CocoAnnTrain.json'), os.path.join(mainPath,'data','train'))
register_coco_instances("my_dataset_val", {}, os.path.join(mainPath,'data','valid','CocoAnnValid.json'), os.path.join(mainPath,'data','valid'))
register_coco_instances("my_dataset_test", {}, os.path.join(mainPath,'data','test','CocoAnnTest.json'), os.path.join(mainPath,'data','test'))

#Training dataset visualization

In [ ]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")
 
import random
from detectron2.utils.visualizer import Visualizer
 
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Train Custom Detectron2 Detector

In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.
 
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
 
class CocoTrainer(DefaultTrainer):
 
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
 
    if output_folder is None:
        
        output_folder = os.path.join(mainPath,'trainedModel','coco_eval')
 
    return COCOEvaluator(dataset_name, cfg, False, output_folder)

##Configuring model for training 
here we configure our custom object detection model 
we chose the FASTER RNN model from the model zoo.
the most important part here is the line **cfg.MODEL.DEVICE='cpu'** where we configure whether we want CPU computation for training or GPU if the goal is GPU the line must be commented out
also for GPU the **cfg.SOLVER.IMS_PER_BATCH**  is very important especially when you get the runtime error. the lower the less memory the GPU uses

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
 
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.001
 
 
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1400 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
 
cfg.MODEL.DEVICE='cpu'
 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 #your number of classes + 1
 
cfg.TEST.EVAL_PERIOD = 500
 
cfg.OUTPUT_DIR = os.path.join(mainPath,"trainedModel")

##Start training 

In [ ]:
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

##Start evaluating the model trained
This function runs an inference on our test set and outputs a table of average Precisions AP which would give us an idea on how the model is performing

In [ ]:
#test evaluation
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir=os.path.join(mainPath,"trainedModel","eval"))
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

# Inference with Detectron2 Saved Weights (saved model)
in this cell, we run inference on the trained model that we have stored in the directory. this cell is independent of the training and when we have a trained model we can use it to do inference on the test set. this cell also outputs three important data structures that are essential to the next part of the algorithm which is detecting the color red

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(mainPath,"trainedModel","model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")
 
imCoordinates = [];
boxPerIm = {}
imname = []
 
for imageName in glob.glob(os.path.join(mainPath,'data','test','*jpg')):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  
  #saving arrays of box coordinates for each image in a list
  instance= outputs["instances"]._fields
  boxes= instance["pred_boxes"]
  boxestensor = boxes.tensor
  boxesArray = boxestensor.cpu().detach().numpy()
  imCoordinates.append(boxesArray)
  #########
  
  ###puting images and their coordinates in a dictionary 
  #boxPerIm[imageName].append(boxesArray)
  boxPerIm.setdefault(imageName,boxesArray)
  imname.append(imageName)

## visualizing the oputput predictions

In [ ]:
for imageName in glob.glob(os.path.join(mainPath,'data','test','*jpg')):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


# Detecting color red

The algorithm:

* 1- the function recives the coordinates of the boundary boxes from the run_inference_on_test_set() function 
* 2- then starts running through the test set directory loading images one by one and cropping the objects out using the boundary box coordinates 
* 3- then saves the cropped image as RGB and counts the number of pixels that have the following RGB value
[x,y,z] 
where 120<x<255 and y<60 and z<60 and abs(y-z)<30 which in my opinion covers most shades of red
* 4- then if these pixels make more than 1 percent of the total pixels (ratio of 0.01) in the cropped image the car is detected 
as red 

note that the mentioned parameters above where set experimental and are opinion based and one might find better ones , i would say finding more systematic way for determining these parameters would be beneficial.but over all the results i would say are average in estimating the car color.

In [ ]:
show_red_car = False ######## if true detected red cars will be displayed
 
i=0;
 
nameList =[]
coordinateList=[]
colorRed=[]
maskList=[]
 
for name in imname:
    coordinates = boxPerIm[name]
    i=0;
    for co in coordinates[:,0]:
        if i<coordinates.shape[0]:
            Wmin =int(coordinates[i][0])
            Hmin =int(coordinates[i][1])
            Wmax =int(coordinates[i][2])
            Hmax =int(coordinates[i][3])
            image = cv2.imread(name)
            image = image[Hmin:Hmax, Wmin:Wmax]
            size = image.shape
            imMask = np.zeros((size[0],size[1])) 
            imlayer = np.zeros((size[0],size[1])) 
            
            ####################
            #showing croping out the bunding box 
            
            # cv2.imshow("cropped", image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            
            ####################
            
            ####estimating the cars color
            c1=0
            c2=0
            
            while c1<size[0]:
                while c2<size[1]:
                    row=image[c1,c2,:]
                    row=row.reshape(-1,1)
                    
                    if 120<int(row[2]) and row[0]<60 and row[1]<60 and abs(row[1]-row[0])<30:
                        imMask[c1,c2] = 255
                    c2=c2+1
                c2=0;
                c1=c1+1
                
            
            nRed = np.count_nonzero(imMask)
            ratio = nRed/(size[0]*size[1])
            
            nameList.append(os.path.basename(name))
            coordinateList.append(str([Wmin,Hmin,Wmax,Hmax]))
            
            if ratio>0.01:
                colorRed.append(True)
                if show_red_car:
                   cv2_imshow(image)    
            else:
                colorRed.append(False)
                        
            #stacking to build the rgb mask#####
            stacked= np.stack((imlayer,imlayer,imMask), axis=2)
            maskList.append(stacked)
            # cv2.imshow('',stacked)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            
            i=i+1;

#Visualizing the image masks 
Following visualizes the resulting RGB image after extracting pixels that matched our criterion for color red in the above cell


In [ ]:
for im in maskList:
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

#CSV output

In [ ]:
finalOut={"file name" : nameList , "box coordinates": coordinateList , "is red": colorRed}; #the final csv out put 
finalOutdf = pd.DataFrame.from_dict(finalOut)
finalOutCSV = finalOutdf.to_csv(index=False)
finalOutdf.to_csv(os.path.join(mainPath,'data','output','finalOutCSV.csv'), index = False)

#Conclusion 

concerning the classification performance of the model when it comes to detecting objects the performance seems acceptable

concerning the color detection performance, the current algorithm works better with higher quality images i would not call the algorithm accurate.
other solutions that may work could be for example considering the average RGB value of the entire image as a metric for detecting color red or training another model on the masks that are generated after applying the initial algorithm.
